In [1]:
import wikipedia
import re
import pandas as pd
import json
from copy import deepcopy

In [2]:
topic_names = ["Health",
              "Environment",
              "Technology",
              "Economy",
              "Entertainment",
              "Sports",
              "Politics",
              "Education",
              "Travel",
              "Food"
]

In [3]:
search_results = {}
for i in range(10):
    search_result = wikipedia.search(topic_names[i], results=1000)
    search_results[topic_names[i]] = search_result
    print(len(search_result),end=", ")

500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 

## Remove non-alphanumeric characters

In [4]:
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

def show_count(search_results):
    for key in search_results:
        print(len(search_results[key]),end=", ")

## Get more pages from embedded links

In [5]:
for key in search_results:
    len_counter = 0
    while len(search_results[key]) <= 1000 and len_counter < len(search_results[key]):
        try:
            content = wikipedia.page(search_results[key][len_counter])#, auto_suggest=False)
        except Exception as e:
            print(e)
            len_counter += 1
            continue
        search_results[key].extend(content.links[:(7-len(search_results[key]))])
        len_counter += 1
show_count(search_results)

/home/pratik/proj1/iir/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/pratik/proj1/iir/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


"Health benefit" may refer to: 
Health benefits (insurance)
Health benefit (medicine)
Health claim
All pages with titles containing Health benefits
"Environment" may refer to: 
Natural environment
Ecology
Environment (systems)
Built environment
Social environment
Market environment
Environment (magazine)
Environment (1917 film)
Environment (1922 film)
Environment (1927 film)
Environments (album series)
Environments (album)
Psychodrama
Environments (journal)
Environment (type theory)
Deployment environment
Runtime environment
Environment variable
Environmentalism
Environmental disease
Environmental health
Environmental science
Environmental history of the United States
Environmental Issues
"Software environment" may refer to: 
Run-time system
Audio synthesis environment
"Environment Secretary" may refer to: 
Cabinet Secretary for Rural Affairs and the Environment
Secretary for the Environment, Transport and Works
Secretary of State for Environment, Food and Rural Affairs
Secretary of th

## Removing duplicate topics

In [10]:
search_results2 = deepcopy(search_results)
for i in range(9):
    for j in range(i+1, 10):
        search_results2[topic_names[j]] = list(set(search_results2[topic_names[j]]) - set(search_results2[topic_names[i]]))
show_count(search_results2)

1226, 1430, 1231, 1125, 985, 1622, 1266, 866, 1038, 1262, 

## Extracting title, revision_id, summary, url and topic from each pages ensuring "summary length" and "preprocessing" requirements

### This script takes 3 hrs to extract contents of all(almost 10000) pages; the output(document {}) has been persisted into a file(document.json) and this section of code is marked as Raw

## Saving to a file

In [5]:
df = pd.read_json("document.json")
df.head(10)

,title,revision_id,summary,url,topic
0,Death,1246577393,Death is the end of life the irreversible cess...,https://en.wikipedia.org/wiki/Death,Health
1,Health (film),1243692893,HealtH also known as Health and H E A L T H is...,https://en.wikipedia.org/wiki/Health_(film),Health
2,Health care,1246665256,Health care or healthcare is the improvement o...,https://en.wikipedia.org/wiki/Health_care,Health
3,Mental health,1247121910,Mental health encompasses emotional psychologi...,https://en.wikipedia.org/wiki/Mental_health,Health
4,World Health Organization,1242561907,The World Health Organization WHO is a special...,https://en.wikipedia.org/wiki/World_Health_Org...,Health
5,Public health,1247277913,Public health is the science and art of preven...,https://en.wikipedia.org/wiki/Public_health,Health
6,Health system,1235311875,A health system health care system or healthca...,https://en.wikipedia.org/wiki/Health_system,Health
7,Health administration,1242061773,Health administration healthcare administratio...,https://en.wikipedia.org/wiki/Health_administr...,Health
8,Occupational safety and health,1242481289,Occupational safety and health OSH or occupati...,https://en.wikipedia.org/wiki/Occupational_saf...,Health
9,Cardinal Health,1245039387,Cardinal Health Inc is an American multination...,https://en.wikipedia.org/wiki/Cardinal_Health,Health


In [6]:
df['topic'].value_counts()

topic
Sports           1385
Environment      1281
Food             1225
Politics         1219
Economy          1174
Technology       1107
Health           1041
Travel           1019
Education         932
Entertainment     794
Name: count, dtype: int64

## Filter documents to around 500 for each categories

In [7]:
original_document = df.to_dict('records')
title_list = {}
for name in topic_names:
    title_list[name] = []
for item in original_document:
    if len(title_list[item['topic']]) < 600:
        title_list[item['topic']].append(item['title'])
title_list[topic_names[0]] = list(set(title_list[topic_names[0]]))
# [1226, 1480, 1321, 1343, 1014, 1680, 1380, 1043, 1239, 1456]
for i in range(9):
    for j in range(i+1, 10):
        title_list[topic_names[j]] = list(set(title_list[topic_names[j]]) - set(title_list[topic_names[i]]))
show_count(title_list)

581, 574, 560, 577, 581, 571, 563, 543, 548, 547, 

In [8]:
filtered_document = []
                
for item in original_document:
    if item['title'] in title_list[item['topic']] and item not in filtered_document:
        filtered_document.append(item)
len(filtered_document)

5645

## Saving to a file

In [10]:
with open('document_final.json', 'w') as json_file:
    json.dump(filtered_document, json_file, indent=4)

### Cleaning titles, lowercasing titles and summary too

In [11]:
for item in filtered_document:
    item['title'] = clean_text(item['title']).lower()
    item['summary'] = item['summary'].lower()

with open('document_final.json', 'w') as json_file:
    json.dump(filtered_document, json_file, indent=4)

In [12]:
df = pd.read_json("document_final.json")
df.head(10)

,title,revision_id,summary,url,topic
0,death,1246577393,death is the end of life the irreversible cess...,https://en.wikipedia.org/wiki/Death,Health
1,health film,1243692893,health also known as health and h e a l t h is...,https://en.wikipedia.org/wiki/Health_(film),Health
2,health care,1246665256,health care or healthcare is the improvement o...,https://en.wikipedia.org/wiki/Health_care,Health
3,mental health,1247121910,mental health encompasses emotional psychologi...,https://en.wikipedia.org/wiki/Mental_health,Health
4,world health organization,1242561907,the world health organization who is a special...,https://en.wikipedia.org/wiki/World_Health_Org...,Health
5,public health,1247277913,public health is the science and art of preven...,https://en.wikipedia.org/wiki/Public_health,Health
6,health system,1235311875,a health system health care system or healthca...,https://en.wikipedia.org/wiki/Health_system,Health
7,health administration,1242061773,health administration healthcare administratio...,https://en.wikipedia.org/wiki/Health_administr...,Health
8,occupational safety and health,1242481289,occupational safety and health osh or occupati...,https://en.wikipedia.org/wiki/Occupational_saf...,Health
9,cardinal health,1245039387,cardinal health inc is an american multination...,https://en.wikipedia.org/wiki/Cardinal_Health,Health


In [13]:
df['topic'].value_counts()

topic
Health           581
Entertainment    581
Economy          577
Environment      574
Sports           571
Politics         563
Technology       560
Travel           548
Food             547
Education        543
Name: count, dtype: int64